In [23]:
#here are some useful imports for this particular series of examples 
from scipy.optimize import linprog
from IPython.core.display import Image, display
! pip install pulp
from __future__ import division
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy

Defaulting to user installation because normal site-packages is not writeable


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-6zjr_1s5 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# 1. Production optimization
- We have four factories, each outputting different products and each with different throughputs and costs to operate the factory per day

- The variables are the number of days each factory should operate to maximize revenues
-The output is the total number of each product that gets produced, which is important for the distribution step to be aware of. 

 The system is constrained in the following ways:
1. Each facotry must meet the goal number of items produced
2. Each factory is limited in the amount of days it can consecutively run

First, we initialize the linear program problem

In [24]:
from pulp import *
problem = LpProblem("problemName", LpMaximize)

Then, we initialize the factory and product parameters

In [25]:
num_factories = 4

# factory cost per day
cost_twinkies = 1e4
cost_hohos = 2e4
cost_cupcakes = 3e4
cost_snowballs = .5e4

# factory throughput per day
produced_twinkies = 2e2
produced_hohos = 1e2
produced_cupcakes = 4e3
produced_snowballs = 5e5


Next, we set production goals and a time limit of number of consecutive days the facotry can run

In [26]:
# production goal
goal_twinkies = 1e3
goal_hohos = 5e2
goal_cupcakes = 1e4
goal_snowballs = 5e5

# time limit
max_num_days = 19

Next, we initialize a dictionary to keep track of the number of days each factory will have to run while also imposing the constraint on the maximum number of days each facotry can consecutively run

In [27]:
factory_days = LpVariable.dicts("factoryDays", list(range(num_factories)), 0, max_num_days, cat="Continuous")
print(factory_days)

{0: factoryDays_0, 1: factoryDays_1, 2: factoryDays_2, 3: factoryDays_3}


Now, we add the constraints
1. Each factory meets the production goal of the item produced
2. Each factory is limited in the amount of days it can consecutively run (already added)

In [28]:
# goal constraint
c1 =factory_days[0]*produced_twinkies >= goal_twinkies
c2= factory_days[1]*produced_hohos >= goal_hohos
c3= factory_days[2] * produced_cupcakes >= goal_cupcakes
c4= factory_days[3] * produced_snowballs >= goal_snowballs

# adding the constraints to the problem
problem += c1
problem += c2
problem += c3
problem += c4

Finally, we form the objective funtion which is to maximize the revenue from the backery production. 

In [29]:
# objective function
problem += (factory_days[0]*cost_twinkies + factory_days[0]*cost_hohos +factory_days[0]*cost_cupcakes +factory_days[0]*cost_snowballs)
print(problem)

problemName:
MAXIMIZE
65000.0*factoryDays_0 + 0.0
SUBJECT TO
_C1: 200 factoryDays_0 >= 1000

_C2: 100 factoryDays_1 >= 500

_C3: 4000 factoryDays_2 >= 10000

_C4: 500000 factoryDays_3 >= 500000

VARIABLES
factoryDays_0 <= 19 Continuous
factoryDays_1 <= 19 Continuous
factoryDays_2 <= 19 Continuous
factoryDays_3 <= 19 Continuous



We now solve the problem and print the sollutions

In [34]:
# Solving
problem.solve()
# Factory throughput per day
solution_twinkies= produced_twinkies * factory_days[0].varValue
solution_hohos=produced_hohos * factory_days[1].varValue
solution_cupcakes=produced_cupcakes * factory_days[2].varValue
solution_snowballs= produced_snowballs * factory_days[3].varValue
solution = [solution_twinkies,solution_hohos,solution_cupcakes,solution_snowballs]
items= ["twinkies","hohos","cupcakes","snowballs"]
for i in range(num_factories):
    print (f" {factory_days[i].varValue} days producing {round(solution[i],0)} {items[i]}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/skyeeiskowitz/.local/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/ab39c56a40ab4c3fb9b40418d0fd272d-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/ab39c56a40ab4c3fb9b40418d0fd272d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 15 RHS
At line 20 BOUNDS
At line 25 ENDATA
Problem MODEL has 4 rows, 4 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-4) rows, 0 (-4) columns and 0 (-4) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 1235000
After Postsolve, objective 1235000, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1235000 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock secon